In [1]:
# curl -X GET -H "Authorization: Bearer <BEARER TOKEN>" "https://api.twitter.com/2/tweets/20"
# bash command to hydrate an example tweet
# get your bearer token (and the keys) here -> https://developer.twitter.com/en/portal/dashboard

In [2]:
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}
credentials['CONSUMER_KEY'] = "xxx"  # key names not consistent ->  API KEY = CONSUMER_KEY
credentials['CONSUMER_SECRET'] = "xxx" # API SECRET KEY = CONSUMER_SECRET
credentials['ACCESS_TOKEN'] = "xxx"   # not needed for now?
credentials['ACCESS_SECRET'] = "xxx" # not needed for now?
credentials['BEARER_TOKEN'] = 'xxx' # seee celll above

# Save the credentials object to file
#with open("twitter_credentials.json", "w") as file:
#    json.dump(credentials, file)
    

In [3]:
#!pip install twython
#!pip install geopy

In [4]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'],credentials['ACCESS_TOKEN'],credentials['ACCESS_SECRET'],credentials['BEARER_TOKEN'])

In [5]:
## show json for a specific ID
from pprint import pprint
tweet= python_tweets.show_status(id="1240727808080412673")
#pprint(tweet)

# path to location. but path varies, so it may fail for other IDS
# -> it fails for every tweet that is not this tweet (see evaluation in next ceel)
# -> TODO better generalization / method
pprint (tweet["extended_entities"]["media"][0]["additional_media_info"]["source_user"]['location'])

'los angeles'


In [9]:
# !pip install tqdm

# request limit: 900 requestis/15 mins.
# so if you make 1 request per second, everything will be fine. (15*60 secs = 900secs)
# but it seems to be a rolling window, so you don't have to wait for a full reset, when limit is reached. 
# adjust sleeping time as required. 
# DONE dynamische Berechnung der time-limits um ensprechend anfragefrequenz automatisch regeln

import time

def calc_delay_to_not_run_into_rate_limit():
    # good if you make requests over long time (>15 mins). if not, shorter delay times then returned here would be ok
    # calculation based on the 'python_tweets.show_status(id)'-method. if we use a batch-request-method other time limits apply.
    python_tweets.get_application_rate_limit_status(resources="statuses")
    resource = python_tweets.get_application_rate_limit_status() # if you get a permession denied error here, you have to change your "project settings" here (developer.twitter.com), to "Read, Write, and Direct Messages" and regenerate all keys. TODO add to readme
    resource = resource["resources"]["statuses"]["/statuses/show/:id"] 
    # -> {'limit': 900, 'remaining': 737, 'reset': 1607445919}
    should_sleep = (resource["reset"] - time.time()) / resource["remaining"]
    return should_sleep # seconds to sleep before next request


import pandas as pd
tweets_csv =  pd.read_csv("data/GeoCOV19TweetsDataset/april1_april2.csv")
tweets_csv.columns = ["ID","Sentiment"] #add column names
del tweets_csv["Sentiment"] # del sentiment column
display (tweets_csv.head()) # display statt print -> print in schön
#print (len(tweets_csv))

from tqdm import tqdm

#iterate over csv files and extract tweet to get location
failed = 0; success = 0; tweet_not_found = 0
for i in tqdm(range(len(tweets_csv))): # ca 600 entries. shorten to len(tweets_csv))-500) for faster execution time
    if i%50 == 0:
        delay = calc_delay_to_not_run_into_rate_limit()
        print ("delay: ",delay)
    time.sleep(delay)            # set to e.g 0.2 if you don't execute this too too often. will be faster and ok. 
    ID = int(tweets_csv['ID'].values[i])
    try:
        tweet= python_tweets.show_status(id=ID)
    except Exception as e: 
        tweet_not_found += 1
        print (e)
        continue
    try:
        print (tweet["extended_entities"]["media"][0]["additional_media_info"]["source_user"]['location'])
    except:
        failed += 1
        continue
    success += 1
print (f"{tweet_not_found}/{len(tweets_csv)} tweets not extracted.\nlocation extraction of extracted tweets: {failed} failed, {success} successful. ")


,ID
0,1245219745311404032
1,1245219922419929088
2,1245221568973848576
3,1245221760246648832
4,1245222299504119808


  0%|          | 0/607 [00:00<?, ?it/s]

delay:  0.7159981795557887


  1%|          | 7/607 [00:06<09:56,  1.01it/s]

Twitter API returned a 404 (Not Found), No status found with that ID.


...

Twitter API returned a 404 (Not Found), No status found with that ID.


 99%|█████████▉| 600/607 [08:06<00:04,  1.44it/s]

delay:  0.4818970142317212


100%|██████████| 607/607 [08:11<00:00,  1.24it/s]

20/607 tweets not extracted.
location extraction: 587 failed, 0 successful. 


In [ ]:
# Create our query
query = {'q': 'corona',
        'count': 10,
        'lang': 'en'
        }


from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="sample app")

# Search tweets
dict_ = {'date': [], 'text': [], 'location': []}
for status in python_tweets.search(**query)['statuses']:
    
    try: 
        if status["user"]["location"] != "":

            dict_['date'].append(status['created_at'])
            dict_['text'].append(status['text'])
            dict_['location'].append(status["user"]["location"])
    except: 
        pass



# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='location', inplace=True, ascending=False)
df.head(5)

for value in dict_['text']:
    print(value)
    print("")
